# Multi-Cycle Experiment Pipeline: Hist-RTSGA



In [ ]:
!pip install openpyxl
# ======================================================
# Cell 1: Imports and Setup
# ======================================================

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Experiment Configuration
RNG_SEED = 42
random.seed(RNG_SEED)
np.random.seed(RNG_SEED)

NUM_CYCLES = 20
RUNS_PER_CYCLE = 30
RTW_RATIOS = [0.05]  # 5% and 10%

# GA Parameters
GA_PARAMS = {
    'max_generations': 100,
    'crossover_prob': 0.8,
    'mutation_prob': 0.05,
    'population_size': 10
}

print("Setup complete. Ready for multi-cycle experiments.")
print(f"Configuration: {NUM_CYCLES} cycles, {RUNS_PER_CYCLE} runs per cycle")
print(f"RTW ratios: {[f'{r*100}%' for r in RTW_RATIOS]}")

Setup complete. Ready for multi-cycle experiments.
Configuration: 20 cycles, 30 runs per cycle
RTW ratios: ['5.0%']


In [ ]:
# ======================================================
# Cell 2: Data Loading and Preparation
# ======================================================

# Load dataset
filename = '/content/drive/MyDrive/RTS/Dataset/mapped_dataset_1-half.xlsx'  # Update this path as needed
df = pd.read_excel(filename)

# Normalize column names
df.columns = [col.strip().lower() for col in df.columns]

# Create data maps
data_maps = {
    'req_to_tests': df.groupby('us_id')['tc_id'].apply(set).to_dict(),
    'req_to_bv': df.groupby('us_id')['us_businessvalue'].first().to_dict(),
    'test_to_time': df.groupby('tc_id')['tc_executiontime'].first().to_dict(),
    'all_req_ids': sorted(df['us_id'].unique()),
    'tests': sorted(df['tc_id'].unique())
}

# For BCPSO: test to requirements mapping
data_maps['test_to_reqs'] = df.groupby('tc_id')['us_id'].apply(set).to_dict()

data_maps['n_reqs'] = len(data_maps['all_req_ids'])
data_maps['n_tests'] = len(data_maps['tests'])

# Calculate total execution time
TOTAL_EXEC_TIME = df.drop_duplicates(subset=['tc_id'])['tc_executiontime'].sum()

print(f"Dataset loaded: {data_maps['n_tests']} tests, {data_maps['n_reqs']} requirements")
print(f"Total execution time: {TOTAL_EXEC_TIME:.2f}")
print(f"Requirements range: {min(data_maps['all_req_ids'])} to {max(data_maps['all_req_ids'])}")
print(f"Business values range: {min(data_maps['req_to_bv'].values())} to {max(data_maps['req_to_bv'].values())}")

Dataset loaded: 40 tests, 124 requirements
Total execution time: 349.65
Requirements range: 1 to 124
Business values range: 2 to 89


In [ ]:
# ======================================================
# Cell 3: Algorithm Implementations (Updated with Starvation Logic)
# ======================================================
import random
import numpy as np

# ======================================================
# The GA Logic as a Reusable Function (Updated)
# ======================================================
def run_ga_instance(max_exec_time, ga_params, data_maps):
    # Unpack GA parameters and data maps
    max_generations = ga_params['max_generations']
    crossover_prob = ga_params['crossover_prob']
    mutation_prob = ga_params['mutation_prob']

    req_to_tests = data_maps['req_to_tests']
    req_to_bv = data_maps['req_to_bv']
    test_to_time = data_maps['test_to_time']
    all_req_ids = data_maps['all_req_ids']
    # bval_to_reqs_map = data_maps['bval_to_reqs_map']  # kept for compatibility if needed
    n_requirements = len(all_req_ids)

    # ----------------------------------------------------------------------
    # New: starvation-related configuration (all optional)
    # ----------------------------------------------------------------------
    # req_to_starvation: requirement -> starvation counter (cycles since last covered)
    req_to_starvation = data_maps.get('req_to_starvation', {})
    # bv_tolerance: fraction of best BV we are willing to accept (e.g., 0.9 = 90%)
    bv_tolerance = ga_params.get('bv_tolerance', 1.0)
    # starvation_weight: strength of starvation bonus when BV is close to best
    starvation_weight = ga_params.get('starvation_weight', 0.0)

    # Track best BV seen (estimated baseline) to apply tolerance inside fitness
    baseline_tracker = {'best_bv_seen': 0.0}

    # ----------------------------------------------------------------------
    # Helper: Evaluation Decomposition (Deb, 2000)
    # Source: Deb (2000), "An Efficient Constraint Handling Method for GAs"
    # ----------------------------------------------------------------------
    def evaluate_chrom(chrom):
        covered_tests = set()
        total_bv = 0
        for idx, val in enumerate(chrom):
            if val:
                req = all_req_ids[idx]
                covered_tests |= req_to_tests.get(req, set())
                total_bv += req_to_bv.get(req, 0)
        total_time = sum(test_to_time.get(t, 0) for t in covered_tests)
        feasible = (total_time <= max_exec_time)
        violation = max(0, total_time - max_exec_time)
        return total_bv, total_time, feasible, violation

    # ----------------------------------------------------------------------
    # New: starvation relief score
    # ----------------------------------------------------------------------
    def starvation_relief(chrom):
        """
        Simple starvation relief: sum of starvation counters
        for all selected requirements. Higher = more relief.
        """
        if not req_to_starvation:
            return 0.0
        relief_score = 0.0
        for idx, val in enumerate(chrom):
            if val:
                req = all_req_ids[idx]
                relief_score += float(req_to_starvation.get(req, 0.0))
        return relief_score

    # ----------------------------------------------------------------------
    # Feasible Initialization (Chu & Beasley, 1998)
    # ----------------------------------------------------------------------
    def feasible_chromosome():
        chrom = [0] * n_requirements
        indices = list(range(n_requirements))
        random.shuffle(indices)
        covered_tests = set()
        total_time = 0
        for idx in indices:
            req = all_req_ids[idx]
            tests = req_to_tests.get(req, set())
            new_tests = tests - covered_tests
            additional_time = sum(test_to_time.get(t, 0) for t in new_tests)
            if total_time + additional_time <= max_exec_time:
                chrom[idx] = 1
                covered_tests |= new_tests
                total_time += additional_time
        return chrom

    population_size = ga_params.get('population_size', 10)
    population = [feasible_chromosome() for _ in range(population_size)]

    # ----------------------------------------------------------------------
    # Greedy Seeding for Budgeted BV Coverage (Khuller, Moss, Naor, 1999)
    # Source: Khuller–Moss–Naor (1999), Budgeted Maximum Coverage (1−1/e)
    # ----------------------------------------------------------------------
    def greedy_seed_variant():
        chrom = [0] * n_requirements
        covered = set()
        used_time = 0
        remaining = set(range(n_requirements))
        while remaining:
            best_idx, best_score = None, float("-inf")
            for idx in remaining:
                req = all_req_ids[idx]
                tests = req_to_tests.get(req, set())
                new_tests = tests - covered
                add_time = sum(test_to_time.get(t, 0) for t in new_tests)
                if add_time < 0:
                    continue
                score = req_to_bv.get(req, 0) if add_time == 0 else (
                    req_to_bv.get(req, 0) / (add_time + 1e-12)
                )
                if used_time + add_time <= max_exec_time and score > best_score:
                    best_idx, best_score = idx, score
            if best_idx is None:
                break
            req = all_req_ids[best_idx]
            new_tests = req_to_tests.get(req, set()) - covered
            add_time = sum(test_to_time.get(t, 0) for t in new_tests)
            if used_time + add_time <= max_exec_time:
                chrom[best_idx] = 1
                covered |= new_tests
                used_time += add_time
            remaining.remove(best_idx)
        return chrom

    # Inject a few seeds to improve the starting population
    num_seeds = min(3, population_size)
    for i in range(num_seeds):
        population[i] = greedy_seed_variant()

    # ----------------------------------------------------------------------
    # Value-Aware Repair / Improve Operator
    # Source: Chu & Beasley (1998); randomized/efficiency-based repair literature
    # ----------------------------------------------------------------------
    def repair_value_aware(chrom):
        chrom = chrom[:]  # work on a copy

        # DROP phase: remove the least efficient requirement until feasible
        while True:
            _, _, feasible, _ = evaluate_chrom(chrom)
            if feasible:
                break
            selected = [i for i, v in enumerate(chrom) if v]
            if not selected:
                break
            worst_idx, worst_ratio = None, float("inf")
            for idx in selected:
                req = all_req_ids[idx]
                covered_other = set()
                for j, v in enumerate(chrom):
                    if v and j != idx:
                        covered_other |= req_to_tests.get(all_req_ids[j], set())
                unique_tests = req_to_tests.get(req, set()) - covered_other
                time_save = sum(test_to_time.get(t, 0) for t in unique_tests)
                bv_loss = req_to_bv.get(req, 0)
                ratio = bv_loss / (time_save + 1e-12) if time_save > 0 else float("inf")
                ratio += random.uniform(-1e-9, 1e-9)  # tie-breaking noise
                if ratio < worst_ratio:
                    worst_ratio, worst_idx = ratio, idx
            if worst_idx is None:
                break
            chrom[worst_idx] = 0

        # ADD phase: greedily add the most efficient requirement that fits
        while True:
            _, tt, _, _ = evaluate_chrom(chrom)
            slack = max_exec_time - tt
            if slack <= 0:
                break
            candidates = [i for i, v in enumerate(chrom) if not v]
            best_idx, best_score = None, float("-inf")
            covered_now = set()
            for j, v in enumerate(chrom):
                if v:
                    covered_now |= req_to_tests.get(all_req_ids[j], set())
            for idx in candidates:
                req = all_req_ids[idx]
                new_tests = req_to_tests.get(req, set()) - covered_now
                add_time = sum(test_to_time.get(t, 0) for t in new_tests)
                if add_time < 0:
                    continue
                score = req_to_bv.get(req, 0) if add_time == 0 else (
                    req_to_bv.get(req, 0) / (add_time + 1e-12)
                )
                score += random.uniform(-1e-9, 1e-9)
                if add_time <= slack and score > best_score:
                    best_idx, best_score = idx, score
            if best_idx is None:
                break
            chrom[best_idx] = 1

        return chrom

    # ----------------------------------------------------------------------
    # Fitness Function (Yoo & Harman, 2012, Sec. 5.1) + starvation extension
    # ----------------------------------------------------------------------
    def fitness(chrom):
        total_bv, _, _, _ = evaluate_chrom(chrom)

        # If no starvation handling configured, behave like original GA.
        if starvation_weight <= 0.0:
            return total_bv

        best_bv_seen = baseline_tracker['best_bv_seen']

        # Only reward starvation when this chromosome's BV is close to best BV.
        if best_bv_seen > 0.0 and bv_tolerance < 1.0:
            min_acceptable_bv = bv_tolerance * best_bv_seen
            if total_bv < min_acceptable_bv:
                # Too far from best BV: ignore starvation bonus.
                return total_bv

        relief_score = starvation_relief(chrom)
        # Combined score: BV is primary, starvation_relief is secondary.
        return total_bv + starvation_weight * relief_score

    # ----------------------------------------------------------------------
    # Deb-Style Tournament Selection (Deb, 2000)
    # ----------------------------------------------------------------------
    def deb_better(a_chrom, b_chrom):
        # Evaluate both chromosomes
        a_bv, _, a_feas, a_viol = evaluate_chrom(a_chrom)
        b_bv, _, b_feas, b_viol = evaluate_chrom(b_chrom)

        # Feasibility first
        if a_feas and not b_feas:
            return True
        if b_feas and not a_feas:
            return False

        if a_feas and b_feas:
            # Both feasible: compare by combined fitness (BV + starvation)
            return fitness(a_chrom) > fitness(b_chrom)

        # Both infeasible: lower violation is better
        return a_viol < b_viol

    def deb_tournament_selection(population, k=3):
        contestants = random.sample(population, k=min(k, len(population)))
        best = contestants[0]
        for c in contestants[1:]:
            if deb_better(c, best):
                best = c
        return best

    # ----------------------------------------------------------------------
    # Crossover and Mutation Operators (Mitchell, 1998) - Unchanged
    # ----------------------------------------------------------------------
    def single_point_crossover(parent1, parent2):
        if n_requirements < 2:
            return parent1[:], parent2[:]
        point = random.randint(1, n_requirements - 1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2

    def mutate(chromosome, mutation_prob=mutation_prob):
        if random.random() < mutation_prob and n_requirements > 0:
            mutation_idx = random.randint(0, n_requirements - 1)
            chromosome[mutation_idx] = 1 - chromosome[mutation_idx]

    # ----------------------------------------------------------------------
    # Initialize baseline best BV from the initial population
    # ----------------------------------------------------------------------
    if population:
        initial_bvs = [evaluate_chrom(ch)[0] for ch in population]
        if initial_bvs:
            baseline_tracker['best_bv_seen'] = max(initial_bvs)

    # ----------------------------------------------------------------------
    # Main GA Loop (modified to use new operators + baseline tracking)
    # ----------------------------------------------------------------------
    for generation in range(max_generations):
        # --- Selection ---
        mating_pool = []
        for _ in range(population_size // 2):
            parent1 = deb_tournament_selection(population)
            parent2 = deb_tournament_selection(population)
            mating_pool.append((parent1, parent2))

        # --- Variation & Repair ---
        offspring = []
        for parent1, parent2 in mating_pool:
            if random.random() < crossover_prob:
                child1, child2 = single_point_crossover(parent1, parent2)
            else:
                child1, child2 = parent1[:], parent2[:]
            mutate(child1)
            mutate(child2)
            child1 = repair_value_aware(child1)
            child2 = repair_value_aware(child2)
            offspring.extend([child1, child2])

        # --- Replacement (using Deb's rules + combined fitness for ranking) ---
        combined_population = population + offspring

        def deb_key(ch):
            total_bv, _, feas, viol = evaluate_chrom(ch)
            if feas:
                # Feasible: prioritize lower violation, then higher combined fitness
                return (0, viol, -fitness(ch))
            # Infeasible: prioritize lower violation, then higher BV
            return (1, viol, -total_bv)

        combined_population.sort(key=deb_key)
        population = combined_population[:population_size]

        # Update best BV seen so far (for BV tolerance)
        best_bv_gen, _, _, _ = evaluate_chrom(population[0])
        if best_bv_gen > baseline_tracker['best_bv_seen']:
            baseline_tracker['best_bv_seen'] = best_bv_gen

    # Final best solution is the top of the population after the last sort
    best_solution = population[0]
    best_fitness, _, _, _ = evaluate_chrom(best_solution)  # this is pure BV
    selected_reqs = [all_req_ids[i] for i, val in enumerate(best_solution) if val]

    return selected_reqs, best_fitness

# ----- HISTORY-AWARE RTSGA (Updated to calculate Starvation) -----
def run_hist_rtsga_cycles(rtw_ratio, data_maps, ga_params, num_cycles=20, runs_per_cycle=30):
    """Run history-aware RTSGA with starvation tracking and sqrt penalty"""
    max_exec_time = rtw_ratio * TOTAL_EXEC_TIME
    req_to_bv_orig = data_maps['req_to_bv']
    all_req_ids = data_maps['all_req_ids']

    # Initialize selection counts (for SQRT penalty)
    select_counts = {req: 0 for req in req_to_bv_orig}

    # Initialize starvation counts (Cycles since last selected)
    # Initially 0 (or you could start higher to encourage early exploration)
    starvation_counts = {req: 0 for req in all_req_ids}

    cycles_results = []

    for cycle_idx in range(1, num_cycles + 1):
        # 1. Apply SQRT PENALTY to BV based on selection history
        req_to_bv_for_cycle = {
            req: req_to_bv_orig[req] / np.sqrt(1 + select_counts[req])
            for req in req_to_bv_orig
        }

        # 2. Prepare Data Maps (include starvation counts)
        data_maps_for_cycle = data_maps.copy()
        data_maps_for_cycle['req_to_bv'] = req_to_bv_for_cycle
        data_maps_for_cycle['req_to_starvation'] = starvation_counts.copy()

        # Run multiple independent runs for this cycle
        all_runs = []
        for run_idx in range(runs_per_cycle):
            selected_reqs, best_fitness = run_ga_instance(
                max_exec_time=max_exec_time,
                ga_params=ga_params,
                data_maps=data_maps_for_cycle
            )
            all_runs.append({
                'selected_reqs': selected_reqs,
                'fitness': best_fitness
            })

        # Find best run
        best_run = max(all_runs, key=lambda r: r['fitness'])

        cycles_results.append({
            'cycle_idx': cycle_idx,
            'best_selected_reqs': best_run['selected_reqs'],
            'best_bv': best_run['fitness'],
            'all_runs': all_runs
        })

        # Update History for next cycle
        selected_set = set(best_run['selected_reqs'])

        for req in all_req_ids:
            if req in selected_set:
                # Selected: Reset starvation, increment selection count
                select_counts[req] += 1
                starvation_counts[req] = 0
            else:
                # Not selected: Increment starvation count
                starvation_counts[req] += 1

    return cycles_results

# ----- STANDARD RTSGA (No History) -----
def run_rtsga_cycles(rtw_ratio, data_maps, ga_params, num_cycles=20, runs_per_cycle=30):
    """Run standard RTSGA without history awareness"""
    max_exec_time = rtw_ratio * TOTAL_EXEC_TIME

    cycles_results = []

    for cycle_idx in range(1, num_cycles + 1):
        # No BV modification - use original values
        all_runs = []
        for run_idx in range(runs_per_cycle):
            selected_reqs, best_fitness = run_ga_instance(
                max_exec_time=max_exec_time,
                ga_params=ga_params,
                data_maps=data_maps
            )
            all_runs.append({
                'selected_reqs': selected_reqs,
                'fitness': best_fitness
            })

        best_run = max(all_runs, key=lambda r: r['fitness'])
        cycles_results.append({
            'cycle_idx': cycle_idx,
            'best_selected_reqs': best_run['selected_reqs'],
            'best_bv': best_run['fitness'],
            'all_runs': all_runs
        })

    return cycles_results

print("Algorithms updated: Hist-RTSGA now tracks starvation.")

Algorithms updated: Hist-RTSGA now tracks starvation.


In [ ]:
# ======================================================
# Cell 4: Multi-Cycle Experiments with Checkpointing
# ======================================================
import time
import os
import pickle
from google.colab import drive

# 1. Define Checkpoint Directory
#    This will create a folder "Starvation_Checkpoints" in your MyDrive.
CHECKPOINT_DIR = "/content/drive/MyDrive/Starvation_Checkpoints1"

if not os.path.exists(CHECKPOINT_DIR):
    os.makedirs(CHECKPOINT_DIR)
    print(f"Created checkpoint directory: {CHECKPOINT_DIR}")
else:
    print(f"Using existing checkpoint directory: {CHECKPOINT_DIR}")

results_storage = {}

# Experiment Configurations
rtw_target = 0.05
bv_tolerances = [0.90, 0.80, 0.70]
starvation_weights = [0.05, 0.10, 0.15, 0.20, 0.25]

print(f"Starting Starvation Sensitivity Experiments at RTW={rtw_target*100}%...")
print(f"Cycles: {NUM_CYCLES}, Runs per Cycle: {RUNS_PER_CYCLE}")

for tol in bv_tolerances:
    for weight in starvation_weights:
        # Create unique label
        config_label = f"Tol{tol}_Wt{weight}"
        checkpoint_file = os.path.join(CHECKPOINT_DIR, f"{config_label}.pkl")

        print(f"\n--- Checking Config: {config_label} ---")

        # --- CHECKPOINT LOGIC ---
        if os.path.exists(checkpoint_file):
            print(f"Found checkpoint for {config_label}. Loading...")
            try:
                with open(checkpoint_file, 'rb') as f:
                    cycle_results = pickle.load(f)
                # Add to current results storage so plotting works later
                results_storage[config_label] = cycle_results
                print("Loaded successfully. Skipping re-computation.")
                continue # Skip to next iteration
            except Exception as e:
                print(f"Error loading checkpoint (file might be corrupted): {e}")
                print("Re-running this configuration...")

        # --- EXECUTION LOGIC (If no checkpoint) ---
        print(f"No checkpoint found. Running Hist-RTSGA...")

        # Update GA Parameters
        current_ga_params = GA_PARAMS.copy()
        current_ga_params['bv_tolerance'] = tol
        current_ga_params['starvation_weight'] = weight

        start_time = time.time()

        # Run the experiment
        cycle_results = run_hist_rtsga_cycles(
            rtw_ratio=rtw_target,
            data_maps=data_maps,
            ga_params=current_ga_params,
            num_cycles=NUM_CYCLES,
            runs_per_cycle=RUNS_PER_CYCLE
        )

        elapsed = time.time() - start_time
        print(f"Completed {config_label} in {elapsed:.2f}s")

        # Save to memory
        results_storage[config_label] = cycle_results

        # Save to Drive (Checkpoint)
        try:
            with open(checkpoint_file, 'wb') as f:
                pickle.dump(cycle_results, f)
            print(f"Checkpoint saved to: {checkpoint_file}")
        except Exception as e:
            print(f"WARNING: Could not save checkpoint: {e}")

print("\n" + "="*70)
print("All 10 combinations completed/loaded successfully!")
print("Keys available in results_storage:", list(results_storage.keys()))
print("="*70)

Using existing checkpoint directory: /content/drive/MyDrive/Starvation_Checkpoints1
Starting Starvation Sensitivity Experiments at RTW=5.0%...
Cycles: 20, Runs per Cycle: 30

--- Checking Config: Tol0.9_Wt0.05 ---
Found checkpoint for Tol0.9_Wt0.05. Loading...
Loaded successfully. Skipping re-computation.

--- Checking Config: Tol0.9_Wt0.1 ---
Found checkpoint for Tol0.9_Wt0.1. Loading...
Loaded successfully. Skipping re-computation.

--- Checking Config: Tol0.9_Wt0.15 ---
Found checkpoint for Tol0.9_Wt0.15. Loading...
Loaded successfully. Skipping re-computation.

--- Checking Config: Tol0.9_Wt0.2 ---
Found checkpoint for Tol0.9_Wt0.2. Loading...
Loaded successfully. Skipping re-computation.

--- Checking Config: Tol0.9_Wt0.25 ---
Found checkpoint for Tol0.9_Wt0.25. Loading...
Loaded successfully. Skipping re-computation.

--- Checking Config: Tol0.8_Wt0.05 ---
Found checkpoint for Tol0.8_Wt0.05. Loading...
Loaded successfully. Skipping re-computation.

--- Checking Config: Tol0.8_Wt0

In [ ]:
# ======================================================
# Cell 5: Create Selection History Tables (Save to Drive & Consolidated)
# ======================================================
import pandas as pd
import os

# 1. Define Directory in Google Drive for Results
RESULTS_DIR = "/content/drive/MyDrive/RTS/Starvation_output_half_size/history_tables"

if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)
    print(f"Created results directory: {RESULTS_DIR}")
else:
    print(f"Using results directory: {RESULTS_DIR}")

def create_selection_history_table(cycles_data, config_name):
    """
    Create table showing (req_id, BV) pairs for the best run in each cycle.
    Adapted for Starvation Sensitivity Analysis structure.
    """
    rows = []

    # We are processing a single configuration's list of cycles
    row = {'Configuration': config_name}

    for cycle_result in cycles_data:
        cycle_idx = cycle_result['cycle_idx']
        best_reqs = cycle_result['best_selected_reqs']

        # Get (req_id, BV) pairs using the global data_maps
        req_bv_pairs = [(req, data_maps['req_to_bv'][req]) for req in best_reqs]

        # Format as string: "(req1, 10), (req5, 50)"
        req_bv_str = ', '.join([f"({r},{bv})" for r, bv in req_bv_pairs])

        row[f'C{cycle_idx}'] = req_bv_str

    rows.append(row)

    df_history = pd.DataFrame(rows)
    return df_history

# 2. Main Processing Loop
print("\nGenerating Selection History Files...")
print("="*60)

# List to collect all dataframes for the consolidated file
all_configs_history_list = []

for config_label in results_storage.keys():
    # Generate table for this specific config
    history_table = create_selection_history_table(results_storage[config_label], config_label)

    # Append to list for consolidated file
    all_configs_history_list.append(history_table)

    # Save individual file to Drive
    history_filename = f'selection_history_{config_label}.xlsx'
    full_path = os.path.join(RESULTS_DIR, history_filename)

    history_table.to_excel(full_path, index=False)
    print(f"Saved Individual: {full_path}")

# 3. Save Consolidated File containing ALL combinations
if all_configs_history_list:
    combined_history_df = pd.concat(all_configs_history_list, ignore_index=True)

    combined_filename = "selection_history_ALL_COMBINED.xlsx"
    combined_full_path = os.path.join(RESULTS_DIR, combined_filename)

    combined_history_df.to_excel(combined_full_path, index=False)

    print("\n" + "="*60)
    print(f"Saved Consolidated File: {combined_full_path}")

print("All selection history tables created successfully!")

Using results directory: /content/drive/MyDrive/RTS/Starvation_output_half_size/history_tables

Generating Selection History Files...
Saved Individual: /content/drive/MyDrive/RTS/Starvation_output_half_size/history_tables/selection_history_Tol0.9_Wt0.05.xlsx
Saved Individual: /content/drive/MyDrive/RTS/Starvation_output_half_size/history_tables/selection_history_Tol0.9_Wt0.1.xlsx
Saved Individual: /content/drive/MyDrive/RTS/Starvation_output_half_size/history_tables/selection_history_Tol0.9_Wt0.15.xlsx
Saved Individual: /content/drive/MyDrive/RTS/Starvation_output_half_size/history_tables/selection_history_Tol0.9_Wt0.2.xlsx
Saved Individual: /content/drive/MyDrive/RTS/Starvation_output_half_size/history_tables/selection_history_Tol0.9_Wt0.25.xlsx
Saved Individual: /content/drive/MyDrive/RTS/Starvation_output_half_size/history_tables/selection_history_Tol0.8_Wt0.05.xlsx
Saved Individual: /content/drive/MyDrive/RTS/Starvation_output_half_size/history_tables/selection_history_Tol0.8_Wt0.1

In [ ]:
# ======================================================
# Cell 6: Calculate Summary Statistics (Save to Drive & Consolidated)
# ======================================================
import numpy as np
import pandas as pd
import os

# 1. Define Directory in Google Drive
RESULTS_DIR = "/content/drive/MyDrive/RTS/Starvation_output_half_size/Statistics"

if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)
    print(f"Using results directory: {RESULTS_DIR}")

# Ensure counts exist in data_maps to avoid KeyError
if 'n_reqs' not in data_maps:
    data_maps['n_reqs'] = len(data_maps['all_req_ids'])
if 'n_tests' not in data_maps:
    data_maps['n_tests'] = len(data_maps['test_to_time'])

def calculate_cycle_statistics(cycles_data, data_maps):
    """
    Calculate mean, std, median for BV, req coverage, and test coverage.
    Input: cycles_data is a LIST of cycle results for one configuration.
    """
    method_stats = []

    for cycle_result in cycles_data:
        cycle_idx = cycle_result['cycle_idx']
        all_runs = cycle_result['all_runs']

        # Extract metrics from all 30 runs
        bv_values = []
        req_cov_values = []
        test_cov_values = []

        for run in all_runs:
            selected_reqs = run['selected_reqs']

            # BV
            total_bv = run['fitness']
            bv_values.append(total_bv)

            # Requirement coverage
            req_cov = (len(selected_reqs) / data_maps['n_reqs']) * 100
            req_cov_values.append(req_cov)

            # Test coverage
            covered_tests = set()
            for req in selected_reqs:
                covered_tests |= data_maps['req_to_tests'].get(req, set())
            test_cov = (len(covered_tests) / data_maps['n_tests']) * 100
            test_cov_values.append(test_cov)

        # Calculate statistics
        cycle_stats = {
            'cycle': cycle_idx,
            'bv_mean': np.mean(bv_values),
            'bv_std': np.std(bv_values),
            'bv_median': np.median(bv_values),
            'bv_max': np.max(bv_values),
            'bv_min': np.min(bv_values),
            'req_cov_mean': np.mean(req_cov_values),
            'req_cov_std': np.std(req_cov_values),
            'req_cov_median': np.median(req_cov_values),
            'req_cov_max': np.max(req_cov_values),
            'req_cov_min': np.min(req_cov_values),
            'test_cov_mean': np.mean(test_cov_values),
            'test_cov_std': np.std(test_cov_values),
            'test_cov_median': np.median(test_cov_values),
            'test_cov_max': np.max(test_cov_values),
            'test_cov_min': np.min(test_cov_values)
        }
        method_stats.append(cycle_stats)

    return pd.DataFrame(method_stats)


def create_summary_table(df_stats, config_name, metric='bv'):
    """Create table with (max, min, std) for each cycle for the specific config"""
    rows = []

    # Row for this specific configuration
    row = {'Configuration': config_name}

    for _, cycle_row in df_stats.iterrows():
        cycle_idx = int(cycle_row['cycle'])
        max_val = cycle_row[f'{metric}_max']
        min_val = cycle_row[f'{metric}_min']
        std_val = cycle_row[f'{metric}_std']

        row[f'C{cycle_idx}'] = f"({max_val:.2f}, {min_val:.2f}, {std_val:.2f})"

    rows.append(row)

    return pd.DataFrame(rows)


# Calculate and save statistics for each configuration
all_statistics = {}

# Lists to hold dataframes for the FINAL consolidated file
consolidated_bv = []
consolidated_req = []
consolidated_test = []

print("Calculating Statistics...")
print("="*60)

for config_label in results_storage.keys():
    print(f"Processing: {config_label}")

    # Calculate stats for this specific configuration list
    df_stats = calculate_cycle_statistics(results_storage[config_label], data_maps)
    all_statistics[config_label] = df_stats

    # Create summary tables for this config
    bv_table = create_summary_table(df_stats, config_label, 'bv')
    req_cov_table = create_summary_table(df_stats, config_label, 'req_cov')
    test_cov_table = create_summary_table(df_stats, config_label, 'test_cov')

    # Append to consolidated lists
    consolidated_bv.append(bv_table)
    consolidated_req.append(req_cov_table)
    consolidated_test.append(test_cov_table)

    # Save INDIVIDUAL file to Drive
    filename = f'summary_statistics_{config_label}.xlsx'
    full_path = os.path.join(RESULTS_DIR, filename)

    with pd.ExcelWriter(full_path, engine='openpyxl') as writer:
        bv_table.to_excel(writer, sheet_name='BV_Summary', index=False)
        req_cov_table.to_excel(writer, sheet_name='Req_Cov_Summary', index=False)
        test_cov_table.to_excel(writer, sheet_name='Test_Cov_Summary', index=False)
        df_stats.to_excel(writer, sheet_name='Full_Statistics', index=False)

    print(f"  -> Saved Individual: {full_path}")

    # Display BV summary (first 6 cycles)
    display_cols = ['Configuration'] + [f'C{i}' for i in range(1, min(7, NUM_CYCLES + 1))]
    print(f"  -> Preview (BV):")
    print(bv_table[display_cols].to_string(index=False))
    print("-" * 40)


# 2. Save CONSOLIDATED file containing ALL combinations
if consolidated_bv:
    combined_bv_df = pd.concat(consolidated_bv, ignore_index=True)
    combined_req_df = pd.concat(consolidated_req, ignore_index=True)
    combined_test_df = pd.concat(consolidated_test, ignore_index=True)

    combined_filename = "summary_statistics_ALL_COMBINED.xlsx"
    combined_path = os.path.join(RESULTS_DIR, combined_filename)

    with pd.ExcelWriter(combined_path, engine='openpyxl') as writer:
        combined_bv_df.to_excel(writer, sheet_name='All_BV_Summaries', index=False)
        combined_req_df.to_excel(writer, sheet_name='All_Req_Summaries', index=False)
        combined_test_df.to_excel(writer, sheet_name='All_Test_Summaries', index=False)

    print("\n" + "="*60)
    print(f"Saved Consolidated File: {combined_path}")

print("\nAll summary statistics calculated and saved successfully!")

Calculating Statistics...
Processing: Tol0.9_Wt0.05
  -> Saved Individual: /content/drive/MyDrive/RTS/Starvation_output_half_size/Statistics/summary_statistics_Tol0.9_Wt0.05.xlsx
  -> Preview (BV):
Configuration                     C1                      C2                      C3                     C4                     C5                     C6
Tol0.9_Wt0.05 (739.00, 711.00, 6.56) (547.89, 502.11, 12.39) (473.34, 429.37, 14.08) (421.44, 410.94, 2.09) (393.92, 365.86, 8.02) (351.95, 326.13, 8.24)
----------------------------------------
Processing: Tol0.9_Wt0.1
  -> Saved Individual: /content/drive/MyDrive/RTS/Starvation_output_half_size/Statistics/summary_statistics_Tol0.9_Wt0.1.xlsx
  -> Preview (BV):
Configuration                     C1                      C2                      C3                     C4                      C5                      C6
 Tol0.9_Wt0.1 (719.00, 711.00, 1.44) (560.11, 519.93, 11.47) (471.55, 439.14, 11.12) (417.42, 398.96, 6.31) (385.42, 346.53, 12

In [ ]:
# ======================================================
# Cell 7: Create Comparative Plots (Fixed Empty Statistics Issue)
# ======================================================
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import pandas as pd
import numpy as np
import os
import pickle
from google.colab import drive

# ------------------------------------------------------
# 1. Force Mount Google Drive
# ------------------------------------------------------
print("Mounting Google Drive...")
try:
    drive.mount('/content/drive')
except Exception as e:
    print("Warning: Could not mount Drive. Files will be saved locally.")

# ------------------------------------------------------
# 2. Configuration & Directories
# ------------------------------------------------------
CHECKPOINT_DIR = "/content/drive/MyDrive/Starvation_Checkpoints1"
RESULTS_DIR = "/content/drive/MyDrive/RTS/Starvation_output_half_size/comparison_plots"

for d in [CHECKPOINT_DIR, RESULTS_DIR]:
    if not os.path.exists(d):
        os.makedirs(d)

# ------------------------------------------------------
# 3. Auto-Recovery: Reload Results if Missing or Empty
# ------------------------------------------------------
# Reload if variable missing OR if it's an empty dictionary
if 'results_storage' not in globals() or not results_storage:
    print("⚠️ 'results_storage' missing or empty. Reloading from checkpoints...")
    results_storage = {}
    if os.path.exists(CHECKPOINT_DIR):
        for f in os.listdir(CHECKPOINT_DIR):
            if f.endswith(".pkl"):
                label = f.replace(".pkl", "")
                try:
                    with open(os.path.join(CHECKPOINT_DIR, f), 'rb') as pkl_file:
                        results_storage[label] = pickle.load(pkl_file)
                except Exception as e:
                    print(f"  - Failed to load {f}: {e}")
    print(f"  -> Recovered {len(results_storage)} configurations.")
else:
    print(f"✅ 'results_storage' found in memory ({len(results_storage)} configs).")

# ------------------------------------------------------
# 4. Auto-Recovery: Recalculate Statistics
# ------------------------------------------------------
# FIXED: Now checks if all_statistics is empty/falsey
if 'all_statistics' not in globals() or not all_statistics:
    print("⚠️ 'all_statistics' missing or empty. Recalculating...")

    if 'data_maps' not in globals():
        print("❌ CRITICAL ERROR: 'data_maps' is missing. Please re-run Cells 1-3 to load your dataset.")
    else:
        # Helper to calculate stats
        def calc_stats_internal(cycles_data):
            method_stats = []
            for cycle_res in cycles_data:
                c_idx = cycle_res['cycle_idx']
                runs = cycle_res['all_runs']
                bv_vals = [r['fitness'] for r in runs]
                req_vals = [(len(r['selected_reqs'])/data_maps['n_reqs'])*100 for r in runs]
                test_vals = []
                for r in runs:
                    cov_t = set()
                    for req in r['selected_reqs']:
                        cov_t |= data_maps['req_to_tests'].get(req, set())
                    test_vals.append((len(cov_t)/data_maps['n_tests'])*100)

                method_stats.append({
                    'cycle': c_idx,
                    'bv_mean': np.mean(bv_vals), 'bv_std': np.std(bv_vals),
                    'req_cov_mean': np.mean(req_vals), 'req_cov_std': np.std(req_vals),
                    'test_cov_mean': np.mean(test_vals), 'test_cov_std': np.std(test_vals)
                })
            return pd.DataFrame(method_stats)

        all_statistics = {}
        for label, data in results_storage.items():
            all_statistics[label] = calc_stats_internal(data)
        print(f"  -> Statistics recalculated for {len(all_statistics)} configs.")
else:
    print(f"✅ 'all_statistics' found in memory ({len(all_statistics)} configs).")

# ------------------------------------------------------
# 5. Load RTSGA Baseline (Updated Path)
# ------------------------------------------------------
rtsga_df = None
found_file = None

# UPDATED: Direct path to your RTSGA Excel file
RTSGA_FILE_PATH = '/content/drive/MyDrive/RTS/Starvation_output_half_size/summary_statistics_RTSGA_only_RTW_5%.xlsx'

# Search candidates (prioritize the new file first)
candidates = [
    RTSGA_FILE_PATH,                                             # Your new RTSGA file (PRIORITY)
    os.path.join(RESULTS_DIR, 'summary_statistics_RTSGA_only_RTW_5%.xlsx'),  # Alternative in RESULTS_DIR
    os.path.join(RESULTS_DIR, 'summary_statistics_RTW_5%.xlsx'), # Old naming
    'summary_statistics_RTSGA_only_RTW_5%.xlsx'                  # Local fallback
]

for path in candidates:
    if os.path.exists(path):
        found_file = path
        break

if found_file:
    try:
        # Load the RTSGA_Full sheet
        rtsga_df = pd.read_excel(found_file, sheet_name='RTSGA_Full')
        print(f"✅ Loaded RTSGA Baseline from: {found_file}")
        print(f"   Data shape: {rtsga_df.shape}")
        print(f"   Columns: {rtsga_df.columns.tolist()}")
    except Exception as e:
        print(f"❌ Error loading file {found_file}: {e}")
        rtsga_df = None
else:
    print("⚠️ WARNING: RTSGA baseline file not found at any of these locations:")
    for path in candidates:
        print(f"  - {path}")
    print("\nPlots will be single-bar only (no baseline comparison).")

# ------------------------------------------------------
# 6. Plotting Logic
# ------------------------------------------------------
def create_comparison_plot(df_hist, df_rtsga, config_name, metric_name, metric_key, ylabel):
    fig, ax = plt.subplots(figsize=(14, 7))

    cycles = df_hist['cycle'].values
    hist_means = df_hist[f'{metric_key}_mean'].values
    hist_stds = df_hist[f'{metric_key}_std'].values
    x = np.arange(len(cycles))
    width = 0.35

    # Bar 1: Current Config (Blue)
    ax.bar(x - width/2, hist_means, width, label=config_name,
           color='#4A90E2', alpha=0.95, yerr=hist_stds, capsize=4)

    # Bar 2: RTSGA Baseline (Orange)
    if df_rtsga is not None:
        limit = min(len(cycles), len(df_rtsga))
        r_means = df_rtsga[f'{metric_key}_mean'].values[:limit]
        r_stds = df_rtsga[f'{metric_key}_std'].values[:limit]
        ax.bar(x[:limit] + width/2, r_means, width, label='RTSGA (Baseline)',
               color='#FF8C42', alpha=0.95, yerr=r_stds, capsize=4)

    ax.set_xlabel('Regression Cycle (k)', fontsize=12, fontweight='bold')
    ax.set_ylabel(ylabel, fontsize=12, fontweight='bold')
    ax.set_title(f'{metric_name}: {config_name} vs RTSGA', fontsize=14, fontweight='bold', pad=15)
    ax.set_xticks(x)
    ax.set_xticklabels([f'C{int(c)}' for c in cycles], fontsize=10)
    ax.legend(loc='upper right', fontsize=11)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    plt.tight_layout()
    return fig

# ------------------------------------------------------
# 7. Generate & Save
# ------------------------------------------------------
print("\nGenerating Comparison Plots...")
pdf_path = os.path.join(RESULTS_DIR, "All_Comparisons_Combined.pdf")
pdf_pages = matplotlib.backends.backend_pdf.PdfPages(pdf_path)

if not all_statistics:
    print("❌ ERROR: Still no statistics available. Please check if checkpoints exist in Drive.")
else:
    for label in all_statistics.keys():
        print(f"  Processing: {label}")
        df_stats = all_statistics[label]

        # BV
        fig1 = create_comparison_plot(df_stats, rtsga_df, label, 'Business Value', 'bv', 'Mean BV')
        fig1.savefig(os.path.join(RESULTS_DIR, f'BV_Compare_{label}.png'), dpi=300)
        pdf_pages.savefig(fig1)
        plt.close(fig1)

        # Req Cov
        fig2 = create_comparison_plot(df_stats, rtsga_df, label, 'Req Coverage', 'req_cov', 'Mean Req Coverage (%)')
        fig2.savefig(os.path.join(RESULTS_DIR, f'ReqCov_Compare_{label}.png'), dpi=300)
        pdf_pages.savefig(fig2)
        plt.close(fig2)

        # Test Cov
        fig3 = create_comparison_plot(df_stats, rtsga_df, label, 'Test Coverage', 'test_cov', 'Mean Test Coverage (%)')
        fig3.savefig(os.path.join(RESULTS_DIR, f'TestCov_Compare_{label}.png'), dpi=300)
        pdf_pages.savefig(fig3)
        plt.close(fig3)

    pdf_pages.close()
    print("\n" + "="*60)
    print(f"✅ Comparison plots saved to: {RESULTS_DIR}")
    print(f"✅ Combined PDF saved as: {pdf_path}")
    print("="*60)

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 'results_storage' found in memory (15 configs).
⚠️ 'all_statistics' missing or empty. Recalculating...
  -> Statistics recalculated for 15 configs.
✅ Loaded RTSGA Baseline from: /content/drive/MyDrive/RTS/Starvation_output_half_size/summary_statistics_RTSGA_only_RTW_5%.xlsx
   Data shape: (20, 16)
   Columns: ['cycle', 'bv_mean', 'bv_std', 'bv_median', 'bv_max', 'bv_min', 'req_cov_mean', 'req_cov_std', 'req_cov_median', 'req_cov_max', 'req_cov_min', 'test_cov_mean', 'test_cov_std', 'test_cov_median', 'test_cov_max', 'test_cov_min']

Generating Comparison Plots...
  Processing: Tol0.9_Wt0.05
  Processing: Tol0.9_Wt0.1
  Processing: Tol0.9_Wt0.15
  Processing: Tol0.9_Wt0.2
  Processing: Tol0.9_Wt0.25
  Processing: Tol0.8_Wt0.05
  Processing: Tol0.8_Wt0.1
  Processing: Tol0.8_Wt0.15
  Processing: Tol0.8_Wt0.2
  Processing: Tol0.8_Wt0.2